# Code Sample

In [1]:
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import hour, col
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql.types import IntegerType

## Initialize Spark Session

In [2]:
# Initialize Spark Session
spark = SparkSession.builder.appName("TaxiDataAnalysis").getOrCreate()
spark.sparkContext.setLogLevel('ERROR')


24/04/23 21:17:32 WARN Utils: Your hostname, dougedemaikebukeaier.local resolves to a loopback address: 127.0.0.1; using 10.0.0.177 instead (on interface en0)
24/04/23 21:17:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/23 21:17:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Set I/O

In [3]:
# Set I/O
local_test = True
if local_test:
    df = spark.read.csv('sample/sample.csv',
                        sep=',', header=True, inferSchema=True)  # .toDF(*header)
    output_folder_path = 'output'
else:
    input_file_path = sys.argv[1]
    df = spark.read.csv(input_file_path, sep=',', header=True, inferSchema=True)
    output_folder_path = sys.argv[2]

print(df.schema)


StructType([StructField('medallion', StringType(), True), StructField('hack_license', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('trip_time_in_secs', IntegerType(), True), StructField('trip_distance', DoubleType(), True), StructField('pickup_longitude', DoubleType(), True), StructField('pickup_latitude', DoubleType(), True), StructField('dropoff_longitude', DoubleType(), True), StructField('dropoff_latitude', DoubleType(), True), StructField('payment_type', StringType(), True), StructField('fare_amount', DoubleType(), True), StructField('surcharge', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('total_amount', DoubleType(), True)])


## Data cleaning

In [4]:
# Data cleaning
df_filtered = df.filter((col('fare_amount') > 0) &
                        (col('trip_distance') > 0) &
                        (col('trip_time_in_secs') > 0))

# Filter out trips where longitude or latitude is 0
df = df_filtered.filter((col('pickup_longitude') != 0) &
                        (col('pickup_latitude') != 0) &
                        (col('dropoff_longitude') != 0) &
                        (col('dropoff_latitude') != 0))

# Filter out trips that happens at least around New York State
nyc_latitude_bounds = (39, 42)
nyc_longitude_bounds = (-76, -72)

df = df.filter(
    (col('pickup_longitude') >= nyc_longitude_bounds[0]) & (col('pickup_longitude') <= nyc_longitude_bounds[1]) &
    (col('pickup_latitude') >= nyc_latitude_bounds[0]) & (col('pickup_latitude') <= nyc_latitude_bounds[1]) &
    (col('dropoff_longitude') >= nyc_longitude_bounds[0]) & (col('dropoff_longitude') <= nyc_longitude_bounds[1]) &
    (col('dropoff_latitude') >= nyc_latitude_bounds[0]) & (col('dropoff_latitude') <= nyc_latitude_bounds[1])
)
print(df.show(5))
print("*** Filter finished ***")

+--------------------+--------------------+-------------------+-------------------+-----------------+-------------+----------------+---------------+-----------------+----------------+------------+-----------+---------+-------+----------+------------+------------+
|           medallion|        hack_license|    pickup_datetime|   dropoff_datetime|trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|payment_type|fare_amount|surcharge|mta_tax|tip_amount|tolls_amount|total_amount|
+--------------------+--------------------+-------------------+-------------------+-----------------+-------------+----------------+---------------+-----------------+----------------+------------+-----------+---------+-------+----------+------------+------------+
|20A85CDF09AC1BE3A...|6AC3258720B5F6600...|2013-06-01 00:00:00|2013-06-01 00:06:00|              360|         0.99|      -74.008789|      40.704243|       -74.017204|       40.708412|         CRD|        6.0|

## Model

In [5]:
# Add time_slot column
df = df.withColumn("time_slot", (hour(col("pickup_datetime")) / 2).cast(IntegerType()))
print("*** Time Slot Added ***")

# Prepare the features for KMeans
vecAssembler = VectorAssembler(inputCols=["pickup_longitude", "pickup_latitude"], outputCol="features")

# Evaluator for computing silhouette score
evaluator = ClusteringEvaluator()

# We will collect the best K and stats in this list
best_ks = []
cluster_stats = []

evaluator = ClusteringEvaluator(predictionCol="cluster", featuresCol="features")

# Loop through each time slot (12 in total)
for time_slot in range(0, 12):
    df_slot = df.filter(col("time_slot") == time_slot)
    df_slot = vecAssembler.transform(df_slot)
    df_slot.cache()
    silhouette_scores = []

    # Apply KMeans for k from 7 to 20 and calculate silhouette scores
    for k in range(7, 21):
        kmeans = KMeans(k=k, seed=1, featuresCol="features", predictionCol="cluster")
        model = kmeans.fit(df_slot)
        predictions = model.transform(df_slot)

        if "cluster" not in predictions.columns:
            raise ValueError("The prediction column 'cluster' was not found in the predictions DataFrame.")

        silhouette = evaluator.evaluate(predictions)
        silhouette_scores.append((k, silhouette))
        print(f"Time Slot {time_slot}: K = {k} and the silhouette = {silhouette}")

    # Find the best k with the highest silhouette score
    best_k = max(silhouette_scores, key=lambda item: item[1])[0]
    best_ks.append((time_slot, best_k))

    print(f"*** The Best K is {best_k} for Time Slot {time_slot} ***")

    # Reapply KMeans with the best k
    best_kmeans = KMeans(k=best_k, seed=1, featuresCol="features", predictionCol="cluster")
    best_model = best_kmeans.fit(df_slot)
    best_predictions = best_model.transform(df_slot)

    # Get centroids of Clusters
    centroids = best_model.clusterCenters()

    # Calculate statistics for each cluster
    for cluster in range(best_k):
        cluster_df = best_predictions.filter(col("cluster") == cluster)

        # Extract centroid coordinates for the current cluster
        centroid_longitude, centroid_latitude = centroids[cluster]

        order_density = cluster_df.count() / (2.0 * 60.0 * 60.0)

        avg_unit_price = cluster_df.groupBy('cluster').avg('fare_amount').first()[1]

        avg_tips = cluster_df.groupBy('cluster').avg('tip_amount').first()[1]

        percentage_tips = cluster_df.withColumn('tip_percentage',
                                                (col('tip_amount') / (
                                                        col('total_amount') - col('tip_amount')))).groupBy(
            'cluster').avg('tip_percentage').first()[1]

        centroid_latitude = float(centroid_latitude)
        centroid_longitude = float(centroid_longitude)

        # Append stats to the list
        cluster_stats.append((time_slot, cluster, centroid_latitude, centroid_longitude, order_density, avg_unit_price,
                              avg_tips, percentage_tips))

        print(f"*** Cluster {cluster} Statistics for Time SLot {time_slot} Calculated***")
        print(cluster_stats)

    df_slot.unpersist()

print("*** Cluster Statistics for Time Slot Calculated ***")

# Convert the best k's to a DataFrame
best_ks_df = spark.createDataFrame(best_ks, schema=["Time_slot", "Best_K"])

# Convert the cluster stats to a DataFrame
stats_schema = ["Time_slot", "Cluster number", "Centroid Latitude", "Centroid Longitude", "Order density",
                "Average unit price per customer", "Average Tips per customer", "Percentage of tips"]

stats_df = spark.createDataFrame(cluster_stats, schema=stats_schema)

# Best k for each time slot
best_ks_df.show()

# Final result
stats_df.show()


*** Time Slot Added ***


Time Slot 0: K = 7 and the silhouette = 0.5064992199093005
Time Slot 0: K = 8 and the silhouette = 0.545110305875861
Time Slot 0: K = 9 and the silhouette = 0.5201905050131413
Time Slot 0: K = 10 and the silhouette = 0.5583138945494259
Time Slot 0: K = 11 and the silhouette = 0.5404954248364835
Time Slot 0: K = 12 and the silhouette = 0.5539472850845616
Time Slot 0: K = 13 and the silhouette = 0.5964565257829996


Time Slot 0: K = 14 and the silhouette = 0.5497093541377305


Time Slot 0: K = 15 and the silhouette = 0.5310592292533893


Time Slot 0: K = 16 and the silhouette = 0.5595477249288543


...

Time Slot 11: K = 16 and the silhouette = 0.598704101222019


Time Slot 11: K = 17 and the silhouette = 0.5910380466666328


Time Slot 11: K = 18 and the silhouette = 0.5467681649939939


Time Slot 11: K = 19 and the silhouette = 0.5559045746467173


Time Slot 11: K = 20 and the silhouette = 0.5716273186099744
*** The Best K is 7 for Time Slot 11 ***
*** Cluster 0 Statistics for Time SLot 11 Calculated***
[(0, 0, 40.74462435160438, -73.98612384138191, 22.03472222222222, 11.330355688622754, 1.2744831389851896, 0.10184788848298101), (0, 1, 40.64666793268913, -73.78398780242962, 2.515277777777778, 44.004998343456656, 4.832184980673661, 0.09501214758590544), (0, 2, 40.81058045346052, -73.95177151792326, 3.1975, 13.230431760924333, 1.1810876552862475, 0.07624586311230577), (0, 3, 40.76781189841654, -73.86922943116433, 2.640277777777778, 25.835632298790113, 3.1299410836401886, 0.10078007886671773), (0, 4, 40.681184586790124, -73.98195912617714, 4.409583333333333, 14.438213172068412, 1.5531188383886094, 0.09597048764836184), (0, 5, 40.71439345948535, -73.95371009988054, 6.520277777777777, 13.30415796873003, 1.5594977207855834, 0.10562304803639404), (0, 6, 40.76303339297546, -73.98672267990727, 21.587222222222223, 12.15344191522763, 1.2493

## Save Clusters

In [6]:
# Save the DataFrame as a single CSV file
stats_df.coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save(
    output_folder_path + "/project-output.csv")

spark.stop()
